# This notebook is for tests of the 1D $P_{F} (k)$ concerning resolution and noise

Please follow the **[this notebook](QA_plots.ipynb)** notebook for general remarks on how to set up the environment at NERSC, **maybe use [the developer edition of the notebook server](jupyter-dev.nersc.gov)** as this runs on a high-memory cori node (and should even allow generating batch jobs on arbitrary nodes from within the notebook) vs. some special thing in the standard version.

## imports

In [5]:
from pathlib import Path
import picca
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline

## set flags and variables

Stuff needed if only a subset of the full catalogue should be used (to speed things up for debugging)

In [41]:
usesubset=True     #should a subset of the full catalogue be used (note that everything takes hours if this is set to False)
rarange=2          #range in ra for subset
decrange=2         #range in dec for subset

Definition of variables needed to define the mocks

In [42]:
mock_base = '/project/projectdirs/desi/mocks/lya_forest/london/v4.0.8/'
quick_ver = 'quick-2.0'

## File/Directory names

Picca installation path and a string to activate the environment for subprocesses

In [43]:
piccadir=Path('/'+'/'.join(Path(picca.__file__).parts[1:-3])+'/')
activate_string='bash -c "source activate picca_plots;'

Catalogue file for the full catalogue

In [44]:
catfile=Path('{}/{}/zcat_desi_drq.fits'.format(mock_base,quick_ver))

Mock files

In [45]:
indir=Path('{}/{}/spectra-16/'.format(mock_base,quick_ver))

Output directories/files for the subset of mocks, the delta fields and Pk

In [46]:
outcatfile=Path.home()/'catalogues_small/'/('_'.join(catfile.parts[-4:])[:-5]+'_reduced_rarange{}_decrange{}.fits'.format(rarange,decrange))
outdir_delta=Path.home()/'res_ascii{}/'.format('_subset_rarange{}_decrange{}' if usesubset else '')
outdir_Pk=Path.home()/'Pk_fits{}/'.format('_subset_rarange{}_decrange{}' if usesubset else '')

Check that output dirs exist (else create)

In [47]:
if usesubset and not outcatfile.parent.exists():
    Path.mkdir(outcatfile.parent,parents=True)
if not outdir_Pk.exists():
    Path.mkdir(outdir_Pk,parents=True)
if not outdir_delta.exists():
    Path.mkdir(outdir_delta,parents=True)

## Create a subset of the mock catalogue

open the mock catalogue

In [48]:
cat=fits.open(catfile)
dec=cat[1].data['DEC']
ra=cat[1].data['RA']

define some range to use

In [49]:
select=(np.abs(dec-40)<decrange)&(np.abs(ra-180)<rarange)

slice the catalogue

In [50]:
if usesubset:
    cat[1].data=cat[1].data[select]

write to a file

In [51]:
if usesubset:
    try:
        cat.writeto(outcatfile)
    except OSError:
        print('File exists, using existing file')

File exists, using existing file


## use picca to compute the delta field

generate the call defined by the directories above

In [52]:
callstring_delta=('{call} --in-dir {indir} --drq {drq} --out-dir {outdir} --mode desi --delta-format Pk1D_ascii --rebin 1 --lambda-min 3650. --lambda-max 7200.0 '
                  '--lambda-rest-min 1050.0 --lambda-rest-max 1180 --nproc 4 --order 0 --use-constant-weight'.format(
                     call=piccadir/'bin/do_deltas.py', indir=indir.as_posix()+'/',drq=(outcatfile if usesubset else catfile).as_posix(),outdir=outdir_delta.as_posix()+'/'))

In [53]:
callstring_delta

'/global/u1/m/mwalther/igmhub/picca/bin/do_deltas.py --in-dir /project/projectdirs/desi/mocks/lya_forest/london/v4.0.8/quick-2.0/spectra-16/ --drq /global/homes/m/mwalther/catalogues_small/london_v4.0.8_quick-2.0_zcat_desi_drq_reduced_rarange2_decrange2.fits --out-dir /global/homes/m/mwalther/res_ascii_subset_rarange{}_decrange{}/ --mode desi --delta-format Pk1D_ascii --rebin 1 --lambda-min 3650. --lambda-max 7200.0 --lambda-rest-min 1050.0 --lambda-rest-max 1180 --nproc 4 --order 0 --use-constant-weight'

---

define an additional string to enable launching subprocesses in the right conda environment

In [54]:
call_delta_full='{} {}"'.format(activate_string,callstring_delta)

use subprocess to actually run the call **do not execute this line unless you're willing to wait for minutes (for subsets) to hours (for full sample)**

TODO: Instead of using subprocess directly maybe use %%sbatch to create a batch script submitted to the queue (this needs some package installations however)

In [55]:
pr_delta=subprocess.run(args=call_delta_full,shell=True,check=True,capture_output=True)

In [56]:
pr_delta.stdout

b' zqso_min = 2.093220338983051\n zqso_max = 5.857142857142857\nmode: desi\n\n start               : nb object in cat = 848\n and thid>0          : nb object in cat = 848\n and ra!=dec         : nb object in cat = 848\n and ra!=0.          : nb object in cat = 848\n and dec!=0.         : nb object in cat = 848\n and z>0.            : nb object in cat = 848\n and z>zmin          : nb object in cat = 544\n and z<zmax          : nb object in cat = 544\nBAL_FLAG_VI not found\n\n\nFound 544 qsos\n\rread 0 of 3. ndata: 0\n\rread 1 of 3. ndata: 75\n\rread 2 of 3. ndata: 166\nfound 544 quasars in input files\n\niteration:  0\ndone\nINFO: using constant weights, skipping eta, var_lss, fudge fits\niteration:  1\ndone\nINFO: using constant weights, skipping eta, var_lss, fudge fits\niteration:  2\ndone\nINFO: using constant weights, skipping eta, var_lss, fudge fits\niteration:  3\ndone\nINFO: using constant weights, skipping eta, var_lss, fudge fits\niteration:  4\ndone\n'

## Compute the power with picca

In [57]:
callstring_Pk=('{call} --in-dir {indir} --in-format ascii --out-dir {outdir} --lambda-obs-min 3750. --noise-estimate pipeline --no-apply-filling'.format(
                     call=piccadir/'bin/do_Pk1D.py', indir=outdir_delta.as_posix(),outdir=outdir_Pk.as_posix()))

In [58]:
callstring_Pk

'/global/u1/m/mwalther/igmhub/picca/bin/do_Pk1D.py --in-dir /global/homes/m/mwalther/res_ascii_subset_rarange{}_decrange{} --in-format ascii --out-dir /global/homes/m/mwalther/Pk_fits_subset_rarange{}_decrange{} --lambda-obs-min 3750. --noise-estimate pipeline --no-apply-filling'

---

define an additional string to enable launching subprocesses in the right conda environment

In [59]:
call_Pk_full='{} {}"'.format(activate_string,callstring_Pk)

use subprocess to actually run the call **do not execute this line unless you're willing to wait for minutes (for subsets) to hours (for full sample)**

In [60]:
pr_Pk=subprocess.run(args=call_Pk_full,shell=True,check=True,capture_output=True)

In [61]:
pr_Pk.stdout

b'\rread 0 of 3 0\n ndata =   67\n\rread 1 of 3 67\n ndata =   147\n\rread 2 of 3 147\n ndata =   469\nall done \n'

## Then plot

In [62]:
callstring_plot='{call} --in-dir {indir} --out-fig {outfile}'.format(call=piccadir/'bin/do_plot_pk1d.py',indir=outdir_Pk, outfile=outdir_Pk/'plot_pk1d.pdf')

In [63]:
callstring_plot

'/global/u1/m/mwalther/igmhub/picca/bin/do_plot_pk1d.py --in-dir /global/homes/m/mwalther/Pk_fits_subset_rarange{}_decrange{} --out-fig /global/homes/m/mwalther/Pk_fits_subset_rarange{}_decrange{}/plot_pk1d.pdf'

---

define an additional string to enable launching subprocesses in the right conda environment

In [64]:
call_plot_full='{} {}"'.format(activate_string,callstring_plot)

use subprocess to actually run the call **do not execute this line unless you're willing to wait for minutes (for subsets) to hours (for full sample)**

In [65]:
pr_plot=subprocess.run(args=call_plot_full,shell=True,check=True,capture_output=True)

In [66]:
pr_plot.stdout

b'\n ndata =   98\n\n ndata =   208\n\n ndata =   706\nFigure(1600x800)\nall done \n'